In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display
from tqdm.notebook import tqdm
pd.options.display.max_columns = None

In [2]:
df = pd.read_excel("./data/CCS_cleaning.xlsx")
df = df.filter(['처방일자#5', '검사결과내용#6'])
display(df)

,처방일자#5,검사결과내용#6
0,2019-08-02,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON..."
1,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ..."
2,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ..."
3,2019-02-26,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ..."
4,2019-02-12,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ..."
...,...,...
31971,2018-04-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM..."
31972,2019-08-13,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON..."
31973,2018-06-21,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM..."
31974,2017-04-20,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM..."


## 1. 정규표현식을 사용하기 전 사전 준비
1. 탐색 대상의 패턴을 파악한다.
2. 탐색 대상을 소문자로 변환한다.
    * 탐색 대상이 대소문자 여부가 중요하지 않은 경우 소문자로 패턴을 정의하는 것이 효율 적이다.


### 탐색 대상의 패턴 확인하기

In [3]:
print(df.loc[0, "검사결과내용#6"])

▣ 검사정보 및 소견

CORONARY CT, CALCIUM SCORE (NON-CONTRAST)

Coronary artery의 calcification 이 있음: 
  LAD, RCA,  Left main artery.

Cardiac chamber dimension의 이상 소견이 없음.

Aortic aneurysm (descending thoracic aorta)이 없음.
Aortic ectasia (ascending thoracic aorta)가 없음.

다른 부위에 석회가 있음: aortic valve,

▣ 결론 및 진단

Coronary artery calcium score:
Degree of plaque burden: moderate, 
AJ 130 = 243
Volume = 226

Percentile rank: >95%.

▣ 의견

임상 관찰 또는 추적 검사

------------------------------------------------------------------------------------------------


In [4]:
print(df.loc[1, "검사결과내용#6"])

▣ 검사정보 및 소견



CORONARY CT, CALCIUM SCORE (NON-CONTRAST)



  LAD와 RCA에 segmental dense calcification들이 있음.

  Descending thoracic aorta에 multiple calcification들이 있음.





▣ 결론 및 진단



CAC score by AJ-130 = 940.56, volume 130 score = 795.92

High degree of plaque burden.

Percentile rank 95% 이상.








위의 데이터의 경우 AJ-130 Score와 Volume Score를 추출하는 것이 목표이다.  

위의 두 샘플을 봤을 때, 샘플 마다 작성하는 방식이 다른 것을 확인 가능하다.  

* Example 
    * AJ 130 = score,  AJ-130 = socre  


* 현재 진행하고 있는 Task에 대해서는 대소문자가 구분이 중요하지 않은 것으로 파악되기 때문에 소문자로 변환해준다.

### 소문자 변환

In [5]:
df["tmp"] = df["검사결과내용#6"].str.lower()
display(df)

,처방일자#5,검사결과내용#6,tmp
0,2019-08-02,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non..."
1,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ..."
2,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ..."
3,2019-02-26,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ..."
4,2019-02-12,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ..."
...,...,...,...
31971,2018-04-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium..."
31972,2019-08-13,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non..."
31973,2018-06-21,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium..."
31974,2017-04-20,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium..."


## 2. 대표적인 패턴 모아서 보기

In [6]:
"""
CAC score by AJ-130 = (score), volume 130 score = (score)

CAC score = 0
CAC = 0

석회수치 51
석회용적 11

석회 수치 = 39
석회 용적 = 37

CAC score by AJ-130 = 1.7.

Calcium volume = 1.7

coronary calcium score = 0.



coronary arteryies에 calcification 없음.

limited evaluation for calcium scoring due to cabg

"""

'\nCAC score by AJ-130 = (score), volume 130 score = (score)\n\nCAC score = 0\nCAC = 0\n\n석회수치 51\n석회용적 11\n\n석회 수치 = 39\n석회 용적 = 37\n\nCAC score by AJ-130 = 1.7.\n\nCalcium volume = 1.7\n\ncoronary calcium score = 0.\n\n\n\ncoronary arteryies에 calcification 없음.\n\nlimited evaluation for calcium scoring due to cabg\n\n'

위의 패턴은 사전이 미리 조사한 패턴이다.  
다양한 패턴이 있으나 정규 표현식을 잘 정의하면 범용적으로 추출할 수 있을 것이다.  
* Example  
    * 석회 용적, 석회용적 => r"석회\s*용적" 으로 단일 패턴으로 기록이 가능하다.

## 3. 패턴 정의하기

* AJ - 130 Score

In [7]:
## Reg_1에 있던 것을 참고하여 패턴을 정의하시오 
aj_0 = re.compile(r"\s*(?P<category>aj[-]130)\s*[=]\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_1 = re.compile(r"\s*cac\s*sc\w+\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_2 = re.compile(r"\s*cac\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_3 = re.compile(r"\s*석회수치\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_4 = re.compile(r"\s*석회\s*수치\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_5 = re.compile(r"\s*aj\s*130\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_6 = re.compile(r"\s*cal\w+\s*s\w+\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_7 = re.compile(r"\s*aj\s*130\s*sc\w*\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")
aj_8 = re.compile(r"\s*n\w*\s*cal\w*\s*in\s*cor\w*\s*art\w*\s*")
aj_9 = re.compile(r"\s*cor\w*\s*ar\w*에\s*cal\w*\s*없\w*\s*")
aj_10 = re.compile(r"\s*aj[-]130\s*sco\w*\s*=\s*(?P<a_score>[0-9]+[.]*[0-9]*)\s*")

* Volume Score

In [8]:
## Reg_1에 있던 것을 참고하여 패턴을 정의하시오 
vl_0 = re.compile(r"\s*volume\s*130\s*score\s*=\s*(?P<v_score>[0-9]+[.]*[0-9]*)\s*")
vl_1 = re.compile(r"\s*석회용적\s*(?P<v_score>[0-9]+[.]*[0-9]*)\s*")
vl_2 = re.compile(r"\s*석회\s*용적\s*=\s*(?P<v_score>[0-9]+[.]*[0-9]*)\s*")
vl_3 = re.compile(r"\s*volume\s*=\s*(?P<v_score>[0-9]+[.]*[0-9]*)\s*")

### 정의한 패턴이 동작하는지 확인

In [9]:
print(df.tmp[0])

▣ 검사정보 및 소견

coronary ct, calcium score (non-contrast)

coronary artery의 calcification 이 있음: 
  lad, rca,  left main artery.

cardiac chamber dimension의 이상 소견이 없음.

aortic aneurysm (descending thoracic aorta)이 없음.
aortic ectasia (ascending thoracic aorta)가 없음.

다른 부위에 석회가 있음: aortic valve,

▣ 결론 및 진단

coronary artery calcium score:
degree of plaque burden: moderate, 
aj 130 = 243
volume = 226

percentile rank: >95%.

▣ 의견

임상 관찰 또는 추적 검사

------------------------------------------------------------------------------------------------


In [10]:
aj = re.search(aj_5, df.loc[0, "tmp"])
vl = re.search(vl_3, df.loc[0, "tmp"])
print("AJ-130 score = ", aj.group("a_score"))
print("Volume 130 score = ", vl.group("v_score") )

AJ-130 score =  243
Volume 130 score =  226


## 4. 모든 Row에 정규 표현식을 적용하여 패턴 찾기 

* AJ-130 Score

In [11]:
## Function 정의
def extract_aj_130_score(input_str):
    if re.search(aj_0, input_str) != None:
        return re.search(aj_0, input_str).group("a_score")
        
    elif re.search(aj_1, input_str) != None:
        return re.search(aj_1, input_str).group("a_score")
        
    elif re.search(aj_2, input_str) != None:
        return re.search(aj_2, input_str).group("a_score")
        
    elif re.search(aj_3, input_str) != None:
        return re.search(aj_3, input_str).group("a_score")
        
    elif re.search(aj_4, input_str) != None:
        return re.search(aj_4, input_str).group("a_score")
        
    elif re.search(aj_5, input_str) != None:
        return re.search(aj_5, input_str).group("a_score")
        
    elif re.search(aj_6, input_str) != None:
        return re.search(aj_6, input_str).group("a_score")
        
    elif re.search(aj_7, input_str) != None:
        return re.search(aj_7, input_str).group("a_score")
        
    elif re.search(aj_8, input_str) != None:
        return "0"
        
    elif re.search(aj_9, input_str) != None:
        return "0"
        
    elif re.search(aj_10, input_str) != None:
        return re.search(aj_10, input_str).group("a_score")
        
    else: return np.nan

In [12]:
## Pandas Apply 사용하기
df['AJT-130 Score'] = df['tmp'].apply(lambda x: extract_aj_130_score(str(x)))

* Volume Score

In [13]:
## Function 정의
def extract_volume_score(input_str):
    if re.search(vl_0, input_str) != None:
        return re.search(vl_0, input_str).group("v_score")
        
    elif re.search(vl_1, input_str) != None:
        return re.search(vl_1, input_str).group("v_score")
        
    elif re.search(vl_2, input_str) != None:
        return re.search(vl_2, input_str).group("v_score")
        
    elif re.search(vl_3, input_str) != None:
        return re.search(vl_3, input_str).group("v_score")
        
    else: return np.nan

In [14]:
## Pandas Apply 사용하기
df['Volume 130 Score'] = df['tmp'].apply(lambda x: extract_volume_score(str(x)))

## 5. 추출 결과 리뷰
* 위의 정규 표현식을 적용시에 숮자 뒤에 . 이 붙는 경우를 확인했다.
    * str의 마지막에 '.'이 있는 경우 float으로 변환 못하기 때문에 str의 마지막 부분이 '.'인 경우 제외하는 후처리를 진행한다.

In [15]:
for i in tqdm(range(df.shape[0])):
    if str(df.loc[i, "AJT-130 Score"])[-1] == ".":
        df.loc[i,"AJT-130 Score"] = str(df.loc[i, "AJT-130 Score"])[0:-1]
    if str(df.loc[i, "Volume 130 Score"])[-1] == ".":
        df.loc[i,"Volume 130 Score"] = str(df.loc[i, "Volume 130 Score"])[0:-1]

  0%|          | 0/31976 [00:00<?, ?it/s]

In [16]:
df["AJT-130 Score"] = df["AJT-130 Score"].astype(float)
df["Volume 130 Score"] = df["Volume 130 Score"].astype(float)

## 5. 최종 결과 확인

In [17]:
display(df)

,처방일자#5,검사결과내용#6,tmp,AJT-130 Score,Volume 130 Score
0,2019-08-02,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non...",243.00,226.00
1,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",940.56,795.92
2,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",0.00,NaN
3,2019-02-26,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",97.00,92.00
4,2019-02-12,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",747.00,256.00
...,...,...,...,...,...
31971,2018-04-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",482.00,399.00
31972,2019-08-13,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non...",0.00,NaN
31973,2018-06-21,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",84.00,24.00
31974,2017-04-20,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",0.00,NaN


In [18]:
df.isnull().sum()

처방일자#5                  0
검사결과내용#6               20
tmp                    20
AJT-130 Score          34
Volume 130 Score    14537
dtype: int64

## 6. 추출되지 않는 경우를 리뷰
* 추출 되지 않는 경우를 봤을 때, 이는 CABG, Stent와 같이 평가가 불가능한 경우인 것을 확인했다.
    * 결측 이유가 있는 경우기 때문에 Reason과 같이 결측 이유를 적는 컬럼을 추가하다.

In [19]:
for i in df[df["AJT-130 Score"].isnull()].index:
    print("\nIndex: ", i)
    print(df.loc[i, "tmp"],"\n")


Index:  198
▣ 검사정보 및 소견



coronary ct, calcium score (non-contrast)



  cabg를 시행 받은 환자로 calcium scoring에 제한적임.





▣ 결론 및 진단



limited evaluation due to cabg.

multiple calcifications at aortic root and descending thoracic aorta.





 


Index:  678
▣ 검사정보 및 소견



coronary ct, calcium score (non-contrast)



  left main, lad, diagonal, rca에 calcification들이 있으며 lcx는 stent가 implantation 되어 있는 것으로 보임.

  lcx의 stent로 인하여 calcium scoring 평가의 제한점이 있음.





▣ 결론 및 진단



s/p pci for lcx.





 


Index:  808
nan 


Index:  884
▣ 검사정보 및 소견



coronary artery ct, calcium score



  coronary artery bypass graft가 시행되어 있는 환자로 calcium scoring에 적합하지 않음.





▣ 결론 및 진단



limited evaluation for calcium scoring due to cabg.





 


Index:  1448
▣ 검사정보 및 소견



coronary artery ct, calcium score



  left main, lad, diagonal, lcx, rca, pda에 multiple dense calcifications이 있음.

  cabg를 시행받은 환자로 calcium score 평가에 제한점이 있음.





▣ 결론 및 진단



s/p cabg.

multiple calcifications in coronary arteries.







In [20]:
reason_re_1 = re.compile(r"\s*(?P<reason>cabg)\s*")
reason_re_2 = re.compile(r"\s*(?P<reason>stent)\s*")

In [21]:
for i in tqdm(df[df["AJT-130 Score"].isnull()].index):
    
    if re.search(reason_re_1, str(df.loc[i, "tmp"])) != None:
        df.loc[i, "Reason"] = re.search(reason_re_1, str(df.loc[i, "tmp"])).group("reason")
    
    elif re.search(reason_re_2, str(df.loc[i, "tmp"])) != None:
        df.loc[i, "Reason"] = re.search(reason_re_2, str(df.loc[i, "tmp"])).group("reason")
    else: continue

display(df)

  0%|          | 0/34 [00:00<?, ?it/s]

,처방일자#5,검사결과내용#6,tmp,AJT-130 Score,Volume 130 Score,Reason
0,2019-08-02,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non...",243.00,226.00,NaN
1,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",940.56,795.92,NaN
2,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",0.00,NaN,NaN
3,2019-02-26,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",97.00,92.00,NaN
4,2019-02-12,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",747.00,256.00,NaN
...,...,...,...,...,...,...
31971,2018-04-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",482.00,399.00,NaN
31972,2019-08-13,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...","▣ 검사정보 및 소견\n\ncoronary ct, calcium score (non...",0.00,NaN,NaN
31973,2018-06-21,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",84.00,24.00,NaN
31974,2017-04-20,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",0.00,NaN,NaN


### 결과 확인

In [22]:
df.query('Reason.notnull()', engine='python')

,처방일자#5,검사결과내용#6,tmp,AJT-130 Score,Volume 130 Score,Reason
198,2019-07-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,cabg
678,2018-08-25,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,stent
884,2017-12-07,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg
1448,2017-04-03,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg
1687,2019-03-25,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,stent
9890,2017-08-17,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent
10557,2018-03-22,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent
10608,2017-05-18,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent
15143,2018-04-09,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent
18359,2016-09-28,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg


## 7. 의뢰자가 원하는 형태로 변환해 주기

In [23]:
for i in df[df["Reason"].notnull()].index:
    if df.loc[i, "Reason"] == "stent":
        df.loc[i, "Note"] = "STENT 시술로 측정 불가"
        df.loc[i, "Comment"] = "STENT 상태로 CAC 측정에 제한적임"
    elif df.loc[i, "Reason"] == "cabg":
        df.loc[i, "Note"] = "CABG 수술로 측정 불가"
        df.loc[i, "Comment"] = "CABG 상태로 CAC 측정이 제한적임"

In [24]:
df.query('Reason.notnull()', engine='python')

,처방일자#5,검사결과내용#6,tmp,AJT-130 Score,Volume 130 Score,Reason,Note,Comment
198,2019-07-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,cabg,CABG 수술로 측정 불가,CABG 상태로 CAC 측정이 제한적임
678,2018-08-25,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
884,2017-12-07,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg,CABG 수술로 측정 불가,CABG 상태로 CAC 측정이 제한적임
1448,2017-04-03,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg,CABG 수술로 측정 불가,CABG 상태로 CAC 측정이 제한적임
1687,2019-03-25,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...","▣ 검사정보 및 소견\n\n\n\ncoronary ct, calcium score ...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
9890,2017-08-17,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
10557,2018-03-22,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
10608,2017-05-18,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
15143,2018-04-09,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,stent,STENT 시술로 측정 불가,STENT 상태로 CAC 측정에 제한적임
18359,2016-09-28,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...","▣ 검사정보 및 소견\n\n\n\ncoronary artery ct, calcium...",NaN,NaN,cabg,CABG 수술로 측정 불가,CABG 상태로 CAC 측정이 제한적임


## 8. 결과 저장하기
* Regular Expression 적용을 위해 임의로 만든 Column을 제거하고서 데이터를 저장하기
* 데이터를 Excel로 저장하거나, DB로 업데이트 하는 방법이 있다. 

In [25]:
df.drop(columns=["Reason", "tmp"], inplace=True)
display(df)

,처방일자#5,검사결과내용#6,AJT-130 Score,Volume 130 Score,Note,Comment
0,2019-08-02,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...",243.00,226.00,NaN,NaN
1,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...",940.56,795.92,NaN,NaN
2,2019-02-15,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...",0.00,NaN,NaN,NaN
3,2019-02-26,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...",97.00,92.00,NaN,NaN
4,2019-02-12,"▣ 검사정보 및 소견\n\n\n\nCORONARY CT, CALCIUM SCORE ...",747.00,256.00,NaN,NaN
...,...,...,...,...,...,...
31971,2018-04-04,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...",482.00,399.00,NaN,NaN
31972,2019-08-13,"▣ 검사정보 및 소견\n\nCORONARY CT, CALCIUM SCORE (NON...",0.00,NaN,NaN,NaN
31973,2018-06-21,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...",84.00,24.00,NaN,NaN
31974,2017-04-20,"▣ 검사정보 및 소견\n\n\n\nCORONARY ARTERY CT, CALCIUM...",0.00,NaN,NaN,NaN
